In [80]:
# !pip install sdv
# !pip install sdmetrics

In [81]:
# !unrar x data.rar

In [82]:
import pandas as pd
import numpy as np
import torch
import random
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sdv.single_table import CTGANSynthesizer as CTGAN
from sdv.metadata import SingleTableMetadata
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import tqdm
from sklearn.metrics import f1_score, r2_score
from xgboost import XGBClassifier, XGBRegressor
from sdmetrics.reports.single_table import QualityReport
#from sdmetrics.metadata import SingleTableMetadata
import pandas as pd

In [83]:
dir_datasets = 'data/original/'

# Загрузка реальных датасетов
real_data_1 = pd.read_csv(dir_datasets+'abalone.csv')
#real_data_1['Rings'] = (real_data_1['Rings'] > 10).astype(int)
real_data_2 = pd.read_csv(dir_datasets+'insurance.csv')
real_data_3 = pd.read_csv(dir_datasets+'my_classification.csv')
real_data_4 = pd.read_csv(dir_datasets+'my_regression.csv')

# Словарь датасетов для удобства
datasets = {
    # 'abolone': {
    #             "data": real_data_1,
    #             "task": "classification",
    #             "target": "Rings",
    #             "num_columns":
    #             ["Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight"]
    #             },
    # 'insurance': {
    #                 "data": real_data_2,
    #                 "task": "regression",
    #                 "target": "expenses",
    #                 "num_columns":
    #                 ["age", "bmi", "children"],
    #                 "cat_columns":
    #                 ["sex", "smoker", "region"]
    #             },
    'my_classification': {
                    "data": real_data_3,
                    "task": "classification",
                    "target": "target",
                    "num_columns":
                    ["feature_0", "feature_1", "feature_2", "feature_3", "feature_4"],
                    "cat_columns":
                    []
                },
    'my_regression': {
                    "data": real_data_4,
                    "task": "regression",
                    "target": "target",
                    "num_columns":
                    ["feature_0", "feature_1", "feature_2", "feature_3"],
                    "cat_columns":
                    []
                },
}

for name, data in datasets.items():
    print(f" Информация о датасете {name}:")
    print(f" Количество строк: {data['data'].shape[0]}")
    print(f" Количество колонок: {data['data'].shape[1]}")
    print(f" Колонки: {list(data['data'].columns)}")
    print(f" Задача: {data['task']}")
    print(f" Целевая переменная: {data['target']}")
    print(f" Числовые признаки: {data['num_columns']}")
    print(f" Категориальные признаки: {data['cat_columns']}")
    print()

 Информация о датасете my_classification:
 Количество строк: 900
 Количество колонок: 6
 Колонки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'target']
 Задача: classification
 Целевая переменная: target
 Числовые признаки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4']
 Категориальные признаки: []

 Информация о датасете my_regression:
 Количество строк: 900
 Количество колонок: 5
 Колонки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'target']
 Задача: regression
 Целевая переменная: target
 Числовые признаки: ['feature_0', 'feature_1', 'feature_2', 'feature_3']
 Категориальные признаки: []



In [84]:
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

In [85]:
def process_data(dataset, num_columns, cat_columns, transformation_num_type='None', transformation_cat_type='None'):

    df_processed = dataset.copy()

    # Обработка числовых признаков
    if transformation_num_type == 'CDF':
        # CDF трансформация: преобразует значения в их эмпирическую функцию распределения
        for col in num_columns:
            # Правильная формула для эмпирической CDF
            df_processed[col] = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

    elif transformation_num_type == 'PLE_CDF':
        # PLE_CDF (Probability Logit Envelope CDF) - более сложная трансформация
        for col in num_columns:
            # Сначала применяем CDF
            cdf_values = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

            # Затем применяем logit трансформацию с небольшим сглаживанием
            # Избегаем 0 и 1 для предотвращения бесконечности в logit
            epsilon = 1e-6
            cdf_values = np.clip(cdf_values, epsilon, 1 - epsilon)

            # Логит трансформация: ln(p/(1-p))
            df_processed[col] = np.log(cdf_values / (1 - cdf_values))

    # Обработка категориальных признаков
    if transformation_cat_type == 'OHE':
        # One Hot Encoding для категориальных признаков
        for col in cat_columns:
            # Создаем dummy переменные с префиксом имени колонки
            dummy_df = pd.get_dummies(df_processed[col], prefix=col, dtype=int)

            # Удаляем исходную категориальную колонку
            df_processed = df_processed.drop(columns=[col])

            # Добавляем новые dummy колонки
            df_processed = pd.concat([df_processed, dummy_df], axis=1)

    return df_processed

In [86]:
ctgan_space = {
    # Learning rates
    'discriminator_lr': hp.qloguniform('discriminator_lr',
                                                 np.log(4e-4), np.log(2.1e-3), 5e-5),
    'generator_lr': hp.qloguniform('generator_lr',
                                             np.log(5e-5), np.log(5e-3), 5e-5),

    # Batch size
    'batch_size': hp.choice('batch_size', [100, 500, 1000]),

    # Embedding dimensions
    'embedding_dim': hp.choice('embedding_dim', [32, 128]),

    # Network architecture
    'generator_dim': hp.choice('generator_dim', [[128, 128, 128], [128, 128, 128, 128]]),
    'discriminator_dim': hp.choice('discriminator_dim', [[256, 256], [256, 256, 256]]),

    # Decay parameters
    'generator_decay': hp.qloguniform('generator_decay',
                                     np.log(1e-6), np.log(6.4e-6), 1e-7),
    'discriminator_decay': hp.qloguniform('discriminator_decay',
                                         np.log(1e-6), np.log(8e-6), 1e-6),

    # Frequency
    'log_frequency': hp.choice('log_frequency', [False, True]),
    'transformation_num_type': hp.choice('transformation_num_type', ['CDF', 'PLE_CDF']),
    'transformation_cat_type': hp.choice('transformation_cat_type', ['OHE'])

}

In [87]:
def evaluate_shape(real_data, synthetic_data, cat_columns):
    """
    Прямое использование метрик SDMetrics без QualityReport.
    Для случаев, когда QualityReport не работает.
    """
    # Подавляем предупреждения SciPy для KS-теста
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="ks_2samp: Exact calculation unsuccessful")

        # Обработка входных данных
        if hasattr(synthetic_data, "dataframe"):
            synthetic = synthetic_data.dataframe()
        else:
            synthetic = synthetic_data.copy()

        real_data = pd.get_dummies(real_data, columns=cat_columns, drop_first=True)
        synthetic = pd.get_dummies(synthetic, columns=cat_columns, drop_first=True)

        # После get_dummies
        real_data = real_data.astype(float)
        synthetic = synthetic.astype(float)

        # Пытаемся использовать прямые метрики
        shape_scores = []

        from sdmetrics.single_column import KSComplement

        for column in real_data.select_dtypes(include=['number']).columns:
            if column in synthetic.columns:
                try:
                    # Проверяем, есть ли достаточно данных и вариативность
                    real_col = real_data[column].dropna()
                    synth_col = synthetic[column].dropna()

                    if len(real_col) > 0 and len(synth_col) > 0:
                        # Проверяем на константные значения
                        if real_col.nunique() > 1 and synth_col.nunique() > 1:
                            ks_score = KSComplement.compute(
                                real_data=real_col,
                                synthetic_data=synth_col
                            )
                            shape_scores.append(ks_score)
                        else:
                            # Для константных колонок используем простую проверку
                            if real_col.iloc[0] == synth_col.iloc[0]:
                                shape_scores.append(1.0)  # Идеальное совпадение
                            else:
                                shape_scores.append(0.0)  # Разные константы

                except Exception as e:
                    print(f"Ошибка при вычислении KS для колонки {column}: {e}")
                    continue

        return sum(shape_scores) / len(shape_scores) if shape_scores else 0.0

In [88]:
from sklearn.model_selection import KFold

def generate_and_evaluate_shape_scores(params, dataset_info, n_splits=3):
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    data = process_data(dataset_info['data'], dataset_info['num_columns'], params['transformation_num_type'])

    kf = KFold(n_splits=n_splits, shuffle=False)
    shape_scores = []

    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index].reset_index(drop=True)
        test_data = data.iloc[test_index].reset_index(drop=True)

        metadata = SingleTableMetadata()
        metadata.detect_from_dataframe(data=train_data)

        # Создание и обучение ctgan с заданными параметрами
        ctgan = CTGAN(
            metadata=metadata,
            discriminator_lr=params['discriminator_lr'],
            generator_lr=params['generator_lr'],
            batch_size=params['batch_size'],
            embedding_dim=params['embedding_dim'],
            generator_dim=params['generator_dim'],
            generator_decay=params['generator_decay'],
            discriminator_decay=params['discriminator_decay'],
            log_frequency=params['log_frequency'],
        )

        ctgan.fit(train_data)

        synthetic_data = ctgan.sample(len(test_data))

        # Вычисляем Shape-score
        score = evaluate_shape(
            test_data,
            synthetic_data,
        dataset_info["cat_columns"])

        shape_scores.append(score)

    return np.mean(shape_scores)

In [89]:
def optimize_dataset(dataset_name, dataset_info, max_evals=30):
    print(f"Оптимизация для датасета: {dataset_name}")

    # Создаем объект для хранения истории поиска
    trials = Trials()

    # Определяем целевую функцию для оптимизации
    def objective(params):
        print("="*50)
        shape_score = generate_and_evaluate_shape_scores(params, dataset_info)

        print(f"Shape-score: {shape_score}")
        print(params)
        print("="*50)

        # Для Shape-score мы хотим максимизировать метрику, поэтому возвращаем отрицательное значение
        return {
            'loss': -shape_score,  # отрицательное для максимизации
            'status': STATUS_OK,
            'shape_score': shape_score
        }

    # Запускаем оптимизацию с помощью TPE алгоритма
    rng = np.random.default_rng(42)  # Используем фиксированный seed
    best = fmin(
        fn=objective,
        space=ctgan_space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=rng
    )

     # Получаем лучшие параметры в читаемом виде
    best_params = {
        'discriminator_lr': best['discriminator_lr'],
        'generator_lr': best['generator_lr'],
        'batch_size': [100, 500, 1000][best['batch_size']],
        'embedding_dim': [32, 128][best['embedding_dim']],
        'generator_dim': [[128, 128, 128], [128, 128, 128, 128]][best['generator_dim']],
        'discriminator_dim': [[256, 256], [256, 256, 256]][best['discriminator_dim']],
        'generator_decay': best['generator_decay'],
        'discriminator_decay': best['discriminator_decay'],
        'log_frequency': [False, True][best['log_frequency']],
        'epochs': float('inf')
    }

    # Находим лучший результат
    best_trial = min(trials.trials, key=lambda x: x['result']['loss'])
    best_shape_score = best_trial['result'].get('shape_score', None)

    results = {
        'best_params': best_params,
        'best_shape_score': best_shape_score,
        'shape_score_diff_from_optimal': -best_trial['result']['loss']  # убираем минус для читаемости
    }

    print(f"Лучший Shape-score для {dataset_name}: {best_shape_score}")
    print(f"Лучшие параметры: {best_params}")
    print('-' * 50)

    return results

In [90]:
# Словарь для хранения результатов
all_results = {}

# Указываем количество итераций для каждого датасета
max_evals = 50

# Запускаем оптимизацию для каждого датасета
for dataset_name in datasets:
    data = datasets[dataset_name]
    all_results[dataset_name] = optimize_dataset(dataset_name, data, max_evals)

Оптимизация для датасета: my_classification
Shape-score: 0.8622222222222223
{'batch_size': 100, 'discriminator_decay': 4e-06, 'discriminator_dim': (256, 256, 256), 'discriminator_lr': 0.0014500000000000001, 'embedding_dim': 128, 'generator_decay': 3.3999999999999996e-06, 'generator_dim': (128, 128, 128), 'generator_lr': 0.0008500000000000001, 'log_frequency': True, 'transformation_cat_type': 'OHE', 'transformation_num_type': 'CDF'}
Shape-score: 0.8383333333333333
{'batch_size': 100, 'discriminator_decay': 6e-06, 'discriminator_dim': (256, 256, 256), 'discriminator_lr': 0.001, 'embedding_dim': 128, 'generator_decay': 1.1e-06, 'generator_dim': (128, 128, 128, 128), 'generator_lr': 0.0008, 'log_frequency': False, 'transformation_cat_type': 'OHE', 'transformation_num_type': 'PLE_CDF'}
Shape-score: 0.8829629629629631
{'batch_size': 1000, 'discriminator_decay': 6e-06, 'discriminator_dim': (256, 256, 256), 'discriminator_lr': 0.0010500000000000002, 'embedding_dim': 128, 'generator_decay': 1.2

In [91]:
for dataset_name in datasets:
    print(dataset_name, all_results[dataset_name]['best_shape_score'])

my_classification 0.8922222222222222
my_regression 0.8857777777777777
